In [1]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
import old.tools, old.RNTK_avg
import jax
import symjax
import symjax.tensor as T
import copy
from RNTK_NEW import RNTK
import time

from RNTK_onediag_dual import RNTK
from RNTK_onediag_dual import create_func as create_func_dual

In [ ]:
import numpy as np

n1 = 10
n2 = 20

Qx = np.ones(n1)
Qxprime = np.ones(n2)
M = np.ones((n1,n2))

def alg2_VT(Qx,Qxprime,M): #K1, K2, K3
    B = np.outer(Qx,Qxprime)
    C = np.sqrt(B)  # in R^{n*n}
    D = M / C  # this is lamblda in ReLU analyrucal formula
    E = np.clip(D, -1, 1)  # clipping E between -1 and 1 for numerical stability.
    F = (1 / (2 * np.pi)) * (E * (np.pi - np.arccos(E)) + np.sqrt(1 - E ** 2)) * C
    G = (np.pi - np.arccos(E)) / (2 * np.pi)
    return F,G

F,G =  VT(Qx,Qxprime,M)

In [38]:
dic = {}
dic["n_patrons1="] = 10#00 #// N (what affect symjax time)
dic["n_entradasTiP="] = 2#00 #// TiPrime length
dic["n_entradasTi="] = 5#00 #// Ti length (multiplier on symjax time)
N = int(dic["n_patrons1="])
ti_length = int(dic["n_entradasTi="])
DATA = T.Placeholder((N, ti_length), 'float32', name = "X")


In [39]:
rntk = RNTK(None, None, None, True)

In [45]:
DATAT[0]

Op(name=_rewriting_take_3, fn=_rewriting_take, shape=(10,), dtype=float32, scope=/default)

In [ ]:
# ok so we have to compute for both T and Tprime at once? but does this mean that they have to be the same length? or was this the T + TP?
def compute_q(T, DATA):
    DATAT = T.transpose(DATA)
    xz = DATAT[0]
    init = rntk.su * 2 * xz*xz[:, None] + rntk.sb**2 + rntk.sh**2

    def scan_func(MINIDATAT, prevq):
        #calculate Q(1, t-1)
            #calculate q(t)
            #calculate q(t')
        return carry, ret_val

In [48]:
n1 = 10
n2 = 20

Qx = np.ones(n1)
Qxprime = np.ones(n2)
M = np.ones((n1,n2))

In [51]:
np.outer(Qx,Qxprime).shape

(10, 20)